In [11]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 3s (89.2 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).


In [12]:
from selenium import webdriver # 웹 브라우저 자동화
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time # 시간 지연
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [13]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.melon.com/chart/week/index.htm?classCd=GN0200'
driver.get(url)
html = driver.page_source
soup = bs(html, 'html.parser')
songs = soup.select('.lst50, .lst100')

In [14]:
data_list = []

In [16]:
for song in songs:
    # 차트 페이지에서의 정보 수집
    title = song.select_one('.ellipsis.rank01 a').text
    rank = song.select_one('.rank').text
    singer = song.select_one('.ellipsis.rank02 a').text
    album = song.select_one('.ellipsis.rank03 a').text
    songid = song.select_one('.like')['data-song-no']

    print(f"Rank: {rank}, Title: {title}, Singer: {singer}, Album: {album}, Song ID: {songid}")

    # 상세 페이지로 이동
    BASE_DETAIL_URL = "https://www.melon.com/song/detail.htm?songId="
    detail_url = BASE_DETAIL_URL + songid
    driver.get(detail_url)
    detail_soup = bs(driver.page_source, 'html.parser')

    # 상세 페이지에서의 정보 수집
    genre = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').find_next_sibling('dd').text.strip()
    release_date = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').text.strip()
    album_image_url = detail_soup.select_one('.thumb .image_typeAll img')['src']
    likes = detail_soup.select_one('#d_like_count').text.strip()
    comments_count = detail_soup.select_one('#revCnt').text.strip()


    # 가사 정보 처리
    lyrics_element = detail_soup.select_one('.lyric')
    if lyrics_element:
        lyrics = lyrics_element.text.strip()
    else:
        lyrics = "Lyrics not found"

    composers = []
    lyricists = []
    arrangers = []



    # 각 li 항목을 반복 처리
    for li in detail_soup.select('.list_person.clfix li'):
      artist_name_el = li.select_one('.ellipsis .artist_name')
      artist_type_el = li.select_one('.meta .type')

      if artist_name_el and artist_type_el:  # 두 항목이 모두 존재할 때만 처리
        artist_name = artist_name_el.text.strip()
        artist_type = artist_type_el.text.strip()

        if artist_type == "작곡":
            composers.append(artist_name)
        elif artist_type == "작사":
            lyricists.append(artist_name)
        elif artist_type == "편곡":
            arrangers.append(artist_name)

    print("Composers:", composers)
    print("Lyricists:", lyricists)
    print("Arrangers:", arrangers)

    data_list.append([songid, title, rank, singer, album, likes, release_date, genre, comments_count, lyrics, album_image_url, ", ".join(composers), ", ".join(lyricists), ", ".join(arrangers)])

Rank: 1, Title: Love Lee, Singer: AKMU (악뮤), Album: Love Lee, Song ID: 36713849
Composers: ['이찬혁', 'MILLENNIUM', 'SIHWANG']
Lyricists: ['이찬혁']
Arrangers: []
Rank: 2, Title: Seven (feat. Latto) - Clean Ver., Singer: 정국, Album: Seven (feat. Latto) - Clean Ver., Song ID: 36617841
Composers: ['Andrew Watt', 'Jon Bellion', 'Henry Walter', 'Theron Makiel Thomas', 'Latto']
Lyricists: ['Andrew Watt', 'Jon Bellion', 'Henry Walter', 'Theron Makiel Thomas', 'Latto']
Arrangers: []
Rank: 3, Title: Super Shy, Singer: NewJeans, Album: NewJeans 2nd EP 'Get Up', Song ID: 36599950
Composers: ['Frankie Scoca', 'Erika de Casier', 'Kristine Bogan']
Lyricists: ['Gigi', '김동현', 'Erika de Casier', 'Kristine Bogan', '다니엘(DANIELLE)']
Arrangers: ['Frankie Scoca']
Rank: 4, Title: ETA, Singer: NewJeans, Album: NewJeans 2nd EP 'Get Up', Song ID: 36635522
Composers: ['250', 'Ylva Dimberg']
Lyricists: ['Gigi', 'Ylva Dimberg', '임성빈']
Arrangers: ['250']
Rank: 5, Title: Fast Forward, Singer: 전소미, Album: GAME PLAN, Song I

In [17]:
# CSV 파일로 저장
import csv
with open('dance_songs.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["SongID", "Title", "Rank", "Singer", "Album", "Likes", "ReleaseDate", "Genre", "CommentsCount", "Lyrics", "AlbumImageURL", "Composers", "Lyricists", "Arrangers"])
    writer.writerows(data_list)